In [ ]:
import wobble
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
mpl.rcParams['xtick.top'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['ytick.minor.size'] = 3
mpl.rcParams['xtick.minor.size'] = 3

In [ ]:
MODEL_T_COLOR = '#1f77b4'
MODEL_STAR_COLOR = '#d62728'
DATA_COLOR = 'k'

In [ ]:
speed_of_light = 2.99792458e8   # m/s

def doppler(v):
    frac = (1. - v/speed_of_light) / (1. + v/speed_of_light)
    return np.sqrt(frac)

## 51 Peg:

In [ ]:
results_51peg = wobble.Results(filename='/Users/mbedell/python/wobble/results/results_51peg_Kstar0_Kt3.hdf5')

### FIGURE: data and models for random epoch

In [ ]:
o = 67 # order
r = np.where(results_51peg.orders == o)[0][0] # index into results to get desired order
e = 7 # epoch
n = np.where(results_51peg.epochs == e)[0][0] # index into results to get desired epoch

In [ ]:
data_51peg = wobble.Data(results_51peg.origin_file, filepath='/Users/mbedell/python/wobble/', 
                   orders=[o], epochs=[e])

In [ ]:
xs = np.exp(data_51peg.xs[0][0])
ys = np.exp(data_51peg.ys[0][0])
mask = data_51peg.ivars[0][0] <= 1.e-8
resids = ys - np.exp(results_51peg.star_ys_predicted[r][n] 
                            + results_51peg.tellurics_ys_predicted[r][n])

In [ ]:
fig, (ax, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[4, 1]}, figsize=(12,5), sharex=True)
ax.scatter(xs, ys, marker=".", alpha=0.5, c=DATA_COLOR, label='data', s=40)
ax.scatter(xs[mask], ys[mask], marker=".", alpha=1., c='white', s=20)
ax.plot(xs, np.exp(results_51peg.star_ys_predicted[r][n]), 
                color=MODEL_STAR_COLOR, label='star model', lw=1.5, alpha=0.8)
ax.plot(xs, np.exp(results_51peg.tellurics_ys_predicted[r][n]), 
                color=MODEL_T_COLOR, label='tellurics model', lw=1.5, alpha=0.8)
ax.set_ylabel('Normalized Flux', fontsize=14)

ax2.scatter(xs, resids, marker=".", alpha=0.5, c=DATA_COLOR, s=40)
ax2.scatter(xs[mask], resids[mask], marker=".", alpha=1., c='white', s=20)
ax2.set_xlabel(r'Wavelength ($\AA$)', fontsize=14)
ax2.set_ylabel('Residuals', fontsize=14)

ax.set_ylim([0.1,1.1])
ax.set_yticks(np.arange(0.2,1.1,0.2))
ax.set_yticks(np.arange(0.1,1.15,0.05), minor=True)
ax.set_xlim([6541,6616])
ax.set_xticks(np.arange(6550, 6611, 10))
ax.set_xticks(np.arange(6541, 6617, 1), minor=True)
ax2.set_ylim([-0.02, 0.02])
ax2.set_yticks(np.arange(-0.02,0.03,0.02))
ax2.set_yticks(np.arange(-0.02,0.02,0.01), minor=True)

ax.legend(fontsize=12)
fig.tight_layout()
fig.subplots_adjust(hspace=0.05)
plt.savefig('51peg_spectrum.png')
plt.savefig('51peg_spectrum.pdf')

### FIGURE: tellurics zoom-in

In [ ]:
o = 63 # order
r = np.where(results_51peg.orders == o)[0][0] # index into results to get desired order
e = 7 # epoch
n = np.where(results_51peg.epochs == e)[0][0] # index into results to get desired epoch

data_51peg = wobble.Data(results_51peg.origin_file, filepath='/Users/mbedell/python/wobble/', 
                   orders=[o], epochs=[e])

xs = np.exp(data_51peg.xs[0][0])
ys = np.exp(data_51peg.ys[0][0])
mask = data_51peg.ivars[0][0] <= 1.e-8
resids = ys - np.exp(results_51peg.star_ys_predicted[r][n] 
                            + results_51peg.tellurics_ys_predicted[r][n])

In [ ]:
fig, (ax, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[4, 1]}, figsize=(12,5), sharex=True)
ax.scatter(xs, ys, marker=".", alpha=0.5, c=DATA_COLOR, label='data', s=40)
ax.scatter(xs[mask], ys[mask], marker=".", alpha=1., c='white', s=20)
ax.plot(xs, np.exp(results_51peg.star_ys_predicted[r][n]), 
                color=MODEL_STAR_COLOR, label='star model', lw=1.5, alpha=0.8)
ax.plot(xs, np.exp(results_51peg.tellurics_ys_predicted[r][n]), 
                color=MODEL_T_COLOR, label='tellurics model', lw=1.5, alpha=0.8)
ax.set_ylabel('Normalized Flux', fontsize=14)

ax2.scatter(xs, resids, marker=".", alpha=0.5, c=DATA_COLOR, s=40)
ax2.scatter(xs[mask], resids[mask], marker=".", alpha=1., c='white', s=20)
ax2.set_xlabel(r'Wavelength ($\AA$)', fontsize=14)
ax2.set_ylabel('Residuals', fontsize=14)

ax.set_ylim([0.35,1.1])
ax.set_yticks(np.arange(0.4,1.1,0.2))
ax.set_yticks(np.arange(0.35,1.15,0.05), minor=True)

ax2.set_ylim([-0.02, 0.02])
ax2.set_yticks(np.arange(-0.02,0.03,0.02))
ax2.set_yticks(np.arange(-0.02,0.02,0.01), minor=True)

ax.set_xlim([6301,6315])
ax.set_xticks(np.arange(6302, 6315, 2))
ax.set_xticks(np.arange(6301, 6315, 0.5), minor=True)

ax.legend(fontsize=12)
fig.tight_layout()
fig.subplots_adjust(hspace=0.05)
plt.savefig('51peg_telluriczoom.png')
plt.savefig('51peg_telluriczoom.pdf')

### FIGURE: orbit fit

In [ ]:
from numpy import log, exp, pi, sqrt, sin, cos, tan, arctan

def calc_ma(T0, t, period):
    # calculate mean anomaly
    days = t - T0
    phase = days/period % 1.0
    ma = phase * 2.0 * pi
    return ma
    
def calc_ea(ma, ecc):
    # calculate eccentric anomaly from mean anomaly, eccentricity
    tolerance = 1e-3
    ea = np.copy(ma)
    while True:
        diff = ea - ecc * sin(ea) - ma
        ea -= diff / (1. - ecc * cos(ea))
        if abs(diff).all() <= tolerance:
            break
    return ea
 
    
def calc_rvs(t,par):
    '''
    Calculate RV(t) given par
    par: [period, K, ecc, omega, M0, offset]
    where omega is the argument of periastron
    and Tp is time at periastron
    '''
    P,K,ecc,omega,tp,offset = par
    
    # enforce boundaries on parameters:
    #if (P < 0.0 or K < 0.0 or ecc < 0.0 or ecc > 0.999 or omega < 0. or omega > 2.*pi or M0 < 0. or M0 > 2.*pi):
    #    return np.zeros_like(t)
    #P = max([0.0, P])
    #K = max([0.0, K])
    #ecc = min([max([0.0, ecc]), 0.99])
    #omega = min([max([-pi, omega]), pi])
    #M0 = min([max([-pi, M0]), pi])
    
    ma = 2. * pi / P * (t - tp)  # mean anomaly
    ea = calc_ea(ma, ecc)  # eccentric anomaly

    f = 2.0 * np.arctan2(sqrt(1+ecc)*sin(ea/2.0), sqrt(1-ecc)*cos(ea/2.0)) # true anomaly
    rvs = - K * (cos(omega + f) + ecc*cos(omega))
    return rvs + offset

def calc_msini(P, K, ecc, Mstar=1.0):
    '''
    works in the limit that msini << Mstar
    takes:
    K - RV semi-amplitude in m/s
    P - period in days
    ecc - eccentricity (dimensionless)
    Mstar - host star mass in solar masses (default 1)
    returns:
    msini - minimum mass in Jupiters
    '''
    scaled_k = np.abs(K) / 28.4329 * np.sqrt(1. - ecc**2)
    msini = scaled_k * (P / 365.)**(1./3.) * Mstar**(2./3.)
    return msini

    
def keplerian(par,x):
    return calc_rvs(x, par)

def resid(par,fn,x,y,yerr):
    model = fn(par,x)
    return (y - model)/yerr 

In [ ]:
rvs = results_51peg.star_time_rvs + results_51peg.bervs
sigs = np.ones_like(rvs) # HACK!!!!
dates = results_51peg.dates - 2450000

In [ ]:
from scipy.optimize import leastsq
par0 = np.asarray([4.2308, 55.65, 0.001, 90 * np.pi/180., 0., 0.])  # [period, K, ecc, omega, tp, offset]
soln = leastsq(resid, par0, args=(keplerian, dates, rvs, sigs))

In [ ]:
par = soln[0]
period = par[0]
date_fold = dates % period
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[4, 1]}, figsize=(8,5), sharex=True)

ax1.errorbar(date_fold/period, rvs - par[-1], sigs, fmt='o')
xs = np.arange(0.,period+0.1,0.1)
ax1.plot(xs/period, calc_rvs(xs, par) - par[-1], color='k')
ax1.set_ylabel(r'RV (m s$^{-1}$)', fontsize=16)

ax2.set_ylim([-4, 4])
ax2.set_xlim([0,1.0])

resids = resid(par,keplerian,dates,rvs,sigs)
ax2.errorbar(date_fold/period, resids, sigs, fmt='o')
ax2.plot(xs/period, np.zeros_like(xs), color='k')
print('chisq = {0:.2f}'.format(np.sum(resids**2/sigs**2)))
print('resids RMS = {0:.2f} m/s'.format(np.std(resids)))
print('planet period = {0:.4f} days'.format(period))
print('planet msini = {0:.2f} MJup'.format(calc_msini(*par[:3])))
ax2.set_ylabel('Resids', fontsize=16)
ax2.set_xlabel('Orbital Phase', fontsize=16)
fig.tight_layout()
fig.subplots_adjust(hspace=.05)
plt.savefig('51peg_planet.png')

## tellurics model 
#### (still using 51 Peg data)

### FIGURE: telluric basis vectors

In [ ]:
r = 63
xlim = [6275,6295]
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[1, 1]}, 
                               figsize=(8,5), sharex=True)
ax1.plot(np.exp(results_51peg.tellurics_template_xs[r]),
       np.exp(results_51peg.tellurics_template_ys[r]), c='k')
for k in range(results_51peg.tellurics_K[r]):
    ax2.plot(np.exp(results_51peg.tellurics_template_xs[r]), 
            np.exp(results_51peg.tellurics_basis_vectors[r][k]) - 0.005*k)
ax1.set_xlim(xlim)
ax1.set_ylim([0.65,1.05])
ax2.set_xlim(xlim)
ax1.set_ylabel('Template Spectrum', fontsize=14)
ax2.set_xlabel(r'Wavelength ($\AA$)', fontsize=14)
ax2.set_ylabel(r'Basis Vectors + $\Delta$', fontsize=14)
ax2.set_yticks(np.arange(0.990,1.005,0.005))
ax2.set_yticks(np.arange(0.987,1.005,0.001), minor=True)
ax2.set_xticks(np.arange(xlim[0],xlim[1]+1,5))
ax2.set_xticks(np.arange(xlim[0],xlim[1],1), minor=True)
fig.tight_layout()
fig.subplots_adjust(hspace=.05)
plt.savefig('telluric_basis.png')

In [ ]:
r = 56
xlim = [5880,5910]
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[1, 1]}, 
                               figsize=(8,5), sharex=True)
ax1.plot(np.exp(results_51peg.tellurics_template_xs[r]),
       np.exp(results_51peg.tellurics_template_ys[r]), c='k')
for k in range(results_51peg.tellurics_K[r]):
    ax2.plot(np.exp(results_51peg.tellurics_template_xs[r]), 
            np.exp(results_51peg.tellurics_basis_vectors[r][k]) - 0.1*k)
ax1.set_xlim(xlim)
ax1.set_ylim([0.9,1.02])
ax2.set_xlim(xlim)
ax1.set_ylabel('Template Spectrum', fontsize=14)
ax2.set_xlabel(r'Wavelength ($\AA$)', fontsize=14)
ax2.set_ylabel(r'Basis Vectors + $\Delta$', fontsize=14)
ax2.set_yticks(np.arange(0.7,1.1,0.1))
ax2.set_yticks(np.arange(0.7,1.1,0.02), minor=True)
ax2.set_xticks(np.arange(xlim[0],xlim[1]+1,5))
ax2.set_xticks(np.arange(xlim[0],xlim[1],1), minor=True)
fig.tight_layout()
fig.subplots_adjust(hspace=.05)
plt.savefig('telluric_basis2.png')

### FIGURE: tellurics compared to standard star

In [ ]:
from wobble.utils import fit_continuum
from astropy.io import fits
r = 63
xlim = [6275,6295]

In [ ]:
spec_file = '/Users/mbedell/python/wobble/data/telluric/HARPS.2009-05-09T23_40_43.280_e2ds_A.fits'
wave_file = '/Users/mbedell/python/wobble/data/telluric/HARPS.2009-05-09T20_24_26.952_wave_A.fits'
sp = fits.open(spec_file)
flux = sp[0].data
sp2 = fits.open(wave_file)
wave = sp2[0].data
snr = sp[0].header['HIERARCH ESO DRS SPE EXT SN{0}'.format(str(int(r)))]

In [ ]:
o = 57
wave2,flux2 = np.exp(results_51peg.tellurics_template_xs[r]), \
              np.exp(results_51peg.tellurics_template_ys[r])
wave1,flux1 = wave[r], flux[r]
ivars1 = np.zeros_like(flux1) + snr**2 # HACK
flux1 = np.exp(np.log(flux1) - fit_continuum(np.log(wave1), np.log(flux1), ivars1))

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.plot(wave1,flux1,color=DATA_COLOR,alpha=0.8,label='telluric standard star')
ax.plot(wave2,flux2,color=MODEL_T_COLOR,alpha=0.8,label='wobble telluric model')
ax.set_xlim(xlim)
ax.set_xlabel(r'Wavelength ($\AA$)', fontsize=16)
ax.set_ylabel('Normalized Flux', fontsize=16)
ax.legend(loc='lower right', fontsize=14)
ax.set_xticks(np.arange(xlim[0],xlim[1]+1,5))
ax.set_xticks(np.arange(xlim[0],xlim[1],1), minor=True)
fig.tight_layout()
fig.savefig('telluric_standard.png')
fig.savefig('telluric_standard.pdf')

## quiet M star:

In [ ]:
results_barnards = wobble.Results(filename='/Users/mbedell/python/wobble/results/results_barnards_Kstar0_Kt0.hdf5')

### FIGURE: data and models for random epoch

In [ ]:
o = 67 # order
r = np.where(results_barnards.orders == o)[0][0] # index into results to get desired order
e = 155 # epoch
n = np.where(results_barnards.epochs == e)[0][0] # index into results to get desired epoch

In [ ]:
data_barnards = wobble.Data(results_barnards.origin_file, filepath='/Users/mbedell/python/wobble/', 
                   orders=[o], epochs=[e])

In [ ]:
xs = np.exp(data_barnards.xs[0][0])
ys = np.exp(data_barnards.ys[0][0])
mask = data_barnards.ivars[0][0] <= 1.e-8
resids = ys - np.exp(results_barnards.star_ys_predicted[r][n] 
                            + results_barnards.tellurics_ys_predicted[r][n])

In [ ]:
fig, (ax, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[4, 1]}, figsize=(12,5), sharex=True)
ax.scatter(xs, ys, marker=".", alpha=0.5, c=DATA_COLOR, label='data', s=40)
ax.scatter(xs[mask], ys[mask], marker=".", alpha=1., c='white', s=20)
ax.plot(xs, np.exp(results_barnards.star_ys_predicted[r][n]), 
                color=MODEL_STAR_COLOR, label='star model', lw=1.5, alpha=0.8)
ax.plot(xs, np.exp(results_barnards.tellurics_ys_predicted[r][n]), 
                color=MODEL_T_COLOR, label='tellurics model', lw=1.5, alpha=0.8)
ax.set_ylabel('Normalized Flux', fontsize=14)

ax2.scatter(xs, resids, marker=".", alpha=0.5, c=DATA_COLOR, s=40)
ax2.scatter(xs[mask], resids[mask], marker=".", alpha=1., c='white', s=20)
ax2.set_xlabel(r'Wavelength ($\AA$)', fontsize=14)
ax2.set_ylabel('Residuals', fontsize=14)

ax.set_ylim([0.15,1.25])
ax.set_yticks(np.arange(0.2,1.4,0.2))
ax.set_yticks(np.arange(0.2,1.2,0.05), minor=True)

ax2.set_ylim([-0.06, 0.06])
ax2.set_yticks(np.arange(-0.05,0.06,0.05))
ax2.set_yticks(np.arange(-0.06,0.06,0.01), minor=True)

ax.set_xlim([6541,6616])
ax.set_xticks(np.arange(6550, 6611, 10))
ax.set_xticks(np.arange(6541, 6617, 1), minor=True)

ax.legend(fontsize=12)
fig.tight_layout()
fig.subplots_adjust(hspace=0.05)
plt.savefig('barnards_spectrum.png')
plt.savefig('barnards_spectrum.pdf')

### FIGURE: tellurics zoom-in

In [ ]:
o = 63 # order
r = np.where(results_barnards.orders == o)[0][0] # index into results to get desired order
e = 155 # epoch
n = np.where(results_barnards.epochs == e)[0][0] # index into results to get desired epoch

data_barnards = wobble.Data(results_barnards.origin_file, filepath='/Users/mbedell/python/wobble/', 
                   orders=[o], epochs=[e])

xs = np.exp(data_barnards.xs[0][0])
ys = np.exp(data_barnards.ys[0][0])
mask = data_barnards.ivars[0][0] <= 1.e-8
resids = ys - np.exp(results_barnards.star_ys_predicted[r][n] 
                            + results_barnards.tellurics_ys_predicted[r][n])

In [ ]:
fig, (ax, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[4, 1]}, figsize=(12,5), sharex=True)
ax.scatter(xs, ys, marker=".", alpha=0.5, c=DATA_COLOR, label='data', s=40)
ax.scatter(xs[mask], ys[mask], marker=".", alpha=1., c='white', s=20)
ax.plot(xs, np.exp(results_barnards.star_ys_predicted[r][n]), 
                color=MODEL_STAR_COLOR, label='star model', lw=1.5, alpha=0.8)
ax.plot(xs, np.exp(results_barnards.tellurics_ys_predicted[r][n]), 
                color=MODEL_T_COLOR, label='tellurics model', lw=1.5, alpha=0.8)
ax.set_ylabel('Normalized Flux', fontsize=14)

ax2.scatter(xs, resids, marker=".", alpha=0.5, c=DATA_COLOR, s=40)
ax2.scatter(xs[mask], resids[mask], marker=".", alpha=1., c='white', s=20)
ax2.set_xlabel(r'Wavelength ($\AA$)', fontsize=14)
ax2.set_ylabel('Residuals', fontsize=14)

ax.set_ylim([0.25,1.3])
ax.set_yticks(np.arange(0.4,1.4,0.2))
ax.set_yticks(np.arange(0.25,1.35,0.05), minor=True)

ax2.set_ylim([-0.06, 0.06])
ax2.set_yticks(np.arange(-0.05,0.06,0.05))
ax2.set_yticks(np.arange(-0.06,0.06,0.01), minor=True)

ax.set_xlim([6301,6315])
ax.set_xticks(np.arange(6302, 6315, 2))
ax.set_xticks(np.arange(6301, 6315, 0.5), minor=True)

ax.legend(fontsize=12)
fig.tight_layout()
fig.subplots_adjust(hspace=0.05)
plt.savefig('barnards_telluriczoom.png')
plt.savefig('barnards_telluriczoom.pdf')

### FIGURE: comparison to PHOENIX model

In [ ]:
from astropy.io import fits
hdul = fits.open('/Users/mbedell/python/wobble/data/lte03300-5.00-0.0.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits')
model_ys = np.copy(hdul[0].data)
hdul = fits.open('/Users/mbedell/python/wobble/data/WAVE_PHOENIX-ACES-AGSS-COND-2011.fits')
model_xs = np.copy(hdul[0].data)

In [ ]:
o = 24 # order
r = np.where(results_barnards.orders == o)[0][0] # index into results to get desired order
e = 155 # epoch
n = np.where(results_barnards.epochs == e)[0][0] # index into results to get desired epoch

data_barnards = wobble.Data(results_barnards.origin_file, filepath='/Users/mbedell/python/wobble/', 
                   orders=[o], epochs=[e])

xs = np.exp(data_barnards.xs[0][0])
ys = np.exp(data_barnards.ys[0][0])
mask = data_barnards.ivars[0][0] <= 1.e-8
resids = ys - np.exp(results_barnards.star_ys_predicted[r][n] 
                            + results_barnards.tellurics_ys_predicted[r][n])

In [ ]:
v_sys = 208.e3 # from eyeballing the Balmer region (4020-4050A)

In [ ]:
fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(111)
ax.plot(model_xs, model_ys/1e13, c='k', ls='--', alpha=0.8, label='PHOENIX model')
ax.plot(xs / doppler(v_sys), np.exp(results_barnards.star_ys_predicted[r][n]), 
                color=MODEL_STAR_COLOR, lw=1.5, alpha=0.8, label='wobble star model')
ax.set_xlim([4450,4465])
ax.set_xticks(np.arange(4450, 4466, 5))
ax.set_xticks(np.arange(4450, 4465, 0.5), minor=True)
ax.set_ylim([0.0,1.45])
ax.set_yticks(np.arange(0.0,1.6,0.2))
ax.set_yticks(np.arange(0.0,1.5,0.05), minor=True)
ax.legend(fontsize=14)

ax.set_xlabel(r'Wavelength ($\AA$)', fontsize=14)
ax.set_ylabel('Normalized Flux', fontsize=14)
plt.savefig('barnards_model.png')
plt.savefig('barnards_model.pdf');

### FIGURE: time series

In [ ]:
fig, (ax, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[4, 1]}, figsize=(8,6))
rvs = results_barnards.star_time_rvs + results_barnards.bervs
pipeline_rvs = results_barnards.pipeline_rvs + results_barnards.bervs
ax.plot(results_barnards.dates, pipeline_rvs, 'r.', alpha=0.5, label='HARPS pipeline')
ax.plot(results_barnards.dates, rvs, 'k.', alpha=0.6, label='wobble')
ax.set_ylabel(r'RV (m s$^{-1}$)', fontsize=16)
ax.set_xticklabels([])

ax2.plot(results_barnards.dates - 2450000, rvs - pipeline_rvs, 'k.', alpha=0.5)
ax2.set_xlabel('MJD', fontsize=16)
ax2.set_ylabel('Residuals', fontsize=16)

ax.legend(fontsize=14)
fig.tight_layout()
fig.subplots_adjust(hspace=0.05)
plt.savefig('barnards_rvs.png')
plt.savefig('barnards_rvs.pdf');